In [1]:
import requests
import pandas as pd
import geopandas as gp
import basal_and_bark
from basal_and_bark import basal_and_bark as basal
import ipyleaflet
from ipyleaflet import Marker, Popup, WidgetControl, GeoData
import ipywidgets
from ipywidgets import HTML, widgets

In [ ]:
states = {"AL": 1,
"AR": 5,
"FL": 12,
"GA": 13,
"KY": 21,
"LA": 22,
"MS": 28,
"NC": 37,
"OK": 40,
"SC": 45,
"TN": 47,
"TX": 48,
"VA": 51
}



In [ ]:
states.keys()

In [ ]:
url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"

In [ ]:
url2 = "https://apps.fs.usda.gov/fiadb-api/fullreport?lat=40.1&lon=84.1&radius=0.5&outputFormat=NJSON"

In [ ]:
# make request
resp = requests.get(url)

In [ ]:

# parse response to json
data = resp.json()

In [ ]:
data.keys()

In [ ]:
data['subtotals'].keys()

In [ ]:


    # create output dictionary and populate it with estimate data frames
outDict = {}
    # append estimates
outDict['estimates'] = pd.DataFrame(data['estimates'])

    # append subtotals and totals if present
if 'subtotals' in data.keys():
     subT = {}
     for i in data['subtotals'].keys():
         subT[i] = pd.DataFrame(data['subtotals'][i])
     outDict['subtotals'] = subT
     outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
outDict['metadata'] = data['metadata']

In [ ]:
outDict["estimates"]

In [2]:
leaflet_map = basal.Map(center = [40,-100], zoom = 4, test = "test", scroll_wheel_zoom = True)

   longitude   latitude                    geometry
0 -83.946533  35.969115  POINT (-83.94653 35.96912)
0    False
Name: geometry, dtype: bool
   longitude   latitude                    geometry
0  -85.12207  35.290469  POINT (-85.12207 35.29047)
0    False
Name: geometry, dtype: bool
   longitude  latitude                    geometry
0 -84.682617  35.43382  POINT (-84.68262 35.43382)
0    False
Name: geometry, dtype: bool
   longitude   latitude                    geometry
0 -84.869385  35.684072  POINT (-84.86938 35.68407)
0    False
Name: geometry, dtype: bool
   longitude   latitude                    geometry
0 -86.220703  36.173357  POINT (-86.22070 36.17336)
0    False
Name: geometry, dtype: bool
   longitude   latitude                    geometry
0 -87.330322  36.146747  POINT (-87.33032 36.14675)
0    False
Name: geometry, dtype: bool
   longitude   latitude                    geometry
0  -88.38501  35.710838  POINT (-88.38501 35.71084)
0    False
Name: geometry, dtype: bool
 

In [3]:
leaflet_map.add_layer_control()

In [4]:
leaflet_map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [6]:
leaflet_map.add_shp("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2018_tn_county.zip")

Map(bottom=1751.0, center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [ ]:
leaflet_map.add_shp("C:/Users/zachd/Downloads/tl_2022_us_state.zip")

In [7]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_widget.clear_output()
basemap_ctrl = WidgetControl(widget=output_widget, position='bottomright')
leaflet_map.add_control(basemap_ctrl)

In [8]:
var = 0

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        #leaflet_map.add_layer(Marker(location=kwargs.get('coordinates')))
        latlon = kwargs.get('coordinates')
        lat = kwargs.get('coordinates')[0]
        lon = kwargs.get('coordinates')[1]

        df = pd.DataFrame({'longitude': [lon], 'latitude': [lat]})

        geometry = gp.points_from_xy(df.longitude, df.latitude, crs="EPSG:4326")

        gdf = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

        tn_counties = gp.read_file("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2018_tn_county.zip")
        tn_counties_gd = gp.GeoDataFrame(tn_counties['geometry'], crs="EPSG:4326")

        tn_counties = tn_counties.set_crs("EPSG:4326", allow_override=True)

        print(gdf)
        print(tn_counties_gd['geometry'][1].contains(gdf['geometry']))
        geo_data = GeoData(geo_dataframe = gdf,
            style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
            hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
            point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3}, name="test")

        leaflet_map.add_layer(geo_data)
        tn_counties_gd = gp.GeoDataFrame(tn_counties, crs="EPSG:4326")
        # tn_counties_gd = gp.GeoSeries(tn_counties, crs="EPSG:4326")
        count = 0
        for i in range(1,len(tn_counties_gd['geometry']),1):
            test = tn_counties_gd['geometry'][i].contains(gdf['geometry'])
            # tn_counties_gd['contain'] = test

            county = 'name'

            if test[0]:
                county = tn_counties_gd['NAME'][i]
                with output_widget:
                    print(county)
            count = count +1

        with output_widget:
            print(df)
            # print(county)
        

leaflet_map.on_interaction(handle_click)

In [ ]:
for i in range(1,len(tn_counties_gd['geometry']),1):
            test = tn_counties_gd['geometry'][i].contains(gdf['geometry'])
            if test[0]==False:
                    print (test)

In [ ]:
match state_name:
    case 'Alabama':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"    
    
    case 'Arkansas':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Florida':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Tennessee':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Kentucky':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Virginia':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'North Carolina':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'South Carolina':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Georgia':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Lousiana':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Texas':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Oklahoma':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"
    
    case 'Mississippi':
        url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=Land%20Use%20-%20Major&cselected=Land%20use&snum=79&wc=102020&outputFormat=NJSON"

In [ ]:
leaflet_map.add_shp("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2018_tn_county.zip")
tn_counties = gp.read_file("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2018_tn_county.zip")

In [ ]:
for i in tn_counties:
    if i['geometry'].contains(geo_data):
        print(i)

In [ ]:
df = pd.DataFrame({'longitude': [-85.151319], 'latitude': [35.252723]})

geometry = gp.points_from_xy(x=df.longitude, y=df.latitude, crs="EPSG:4326")

gdf = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")

tn_counties = gp.read_file("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2018_tn_county.zip")

#-86.462284  35.603719
#35.252723, -85.151319

In [ ]:
#tn_counties = gp.read_file("https://github.com/ZachDorm/basal_and_bark/raw/main/docs/examples/data/tl_2018_tn_county.zip")
tn_counties_gd = gp.GeoDataFrame(tn_counties, crs="EPSG:4326")
tn_counties_gs = gp.GeoSeries(tn_counties['geometry'], crs="EPSG:4326")

tn_counties = tn_counties.set_crs("EPSG:4326", allow_override=True)


In [ ]:
tn_counties_gd

In [ ]:
tn_counties

In [ ]:
gdf

In [ ]:
def testIt():
    test = tn_counties_gd.contains(gdf['geometry'])
    tn_counties_gd['contain'] = test
    count = 0
    for i in tn_counties_gd['contain']:
        if i:
            print(type(tn_counties_gd['NAME'][count]))
        count = count +1
    # contained = tn_counties_gd.loc[tn_counties_gd['contain']]==True
# print(tn_counties_gd.loc[tn_counties_gd['contain']]==True)